In [1]:
import sqlalchemy
from sqlalchemy import text, select

import cv2 as cv
import numpy as np
from matplotlib import pyplot as plt
from ipywidgets import widgets, interactive, interact

from watcher import *
from watcher.video import *
from bounding_box_exp import find_moving_objects, draw_boxes

In [10]:
query_most_recent_events = """
select event_name from event_observations obs
where obs.storage_local is True
and obs.lighting_type = 'daylight'
order by capture_time desc
limit 50
"""

tc = TunneledConnection()

session = sqlalchemy.orm.Session(tc.connect())
events = session.query().from_statement(text(query_most_recent_events)).fetchall()

events

AttributeError: 'Query' object has no attribute 'fetchall'

In [12]:
event_data = {}
ev = None
boxes = None
plt.rcParams['figure.figsize'] = [20, 10]    

In [13]:
@interact(event_name=recent_events)
def set_evt(event_name):
    global ev, boxes
    ev = event_data.get(event_name, EventVideo(name = event_name))
    ev.load_frames();
    boxes = find_moving_objects(ev.file, show_work=False)
    event_data[event_name] = ev
    
@interact(frame=(0, ev.num_frames))
def show_frame(frame=ev.most_significant_frame()):
    global boxes
    
    frame_out = ev.frames[frame,:,:,:]
    frame_out = draw_boxes(frame_out, boxes[frame])
    frame_out = draw_boxes(frame_out, ev.event.boxes_for_frame(frame - 100), (255,0,0))
    plt.imshow(frame_out)
    
    for m in sorted(ev.event.motions, key=lambda m: m.frame):
        print(f"frame: {m.frame} box: {m.box()}")

interactive(children=(Dropdown(description='event_name', options=('20220624_143148_wellerDriveway_16', '202206…

interactive(children=(IntSlider(value=106, description='frame', max=165), Output()), _dom_classes=('widget-int…

In [ ]:
boxes